In [9]:
import pandas as pd

df = pd.read_csv("schneider50k.tsv",sep="\t")

# original_rxn: the mapped reaction SMILES (to be transformed for kernels)
# rxn_class: the classification class of the reaction (to be learned)

# Remove source
df.drop(columns=["source"], inplace=True)

#print(df.head())
print(df.iloc[0]["original_rxn"])

[CH3:17][S:14](=[O:15])(=[O:16])[N:11]1[CH2:10][CH2:9][N:8](Cc2ccccc2)[CH2:13][CH2:12]1.C1CCCCC1>[OH-].[OH-].[Pd+2].CCO>[CH3:17][S:14](=[O:15])(=[O:16])[N:11]1[CH2:10][CH2:9][NH:8][CH2:13][CH2:12]1


In [6]:
# Categorize by rxn_class and count
class_counts = df['rxn_class'].value_counts()
print(class_counts)

rxn_class
1.2.1     1000
1.2.4     1000
1.2.5     1000
1.3.6     1000
1.3.7     1000
1.3.8     1000
1.6.2     1000
1.6.4     1000
1.6.8     1000
1.7.4     1000
1.7.6     1000
1.7.7     1000
1.7.9     1000
1.8.5     1000
10.1.1    1000
10.1.2    1000
10.1.5    1000
10.2.1    1000
10.4.2    1000
2.1.1     1000
2.1.2     1000
2.1.7     1000
2.2.3     1000
2.3.1     1000
2.6.1     1000
2.6.3     1000
2.7.2     1000
3.1.1     1000
3.1.5     1000
3.1.6     1000
3.3.1     1000
3.4.1     1000
5.1.1     1000
6.1.1     1000
6.1.3     1000
6.1.5     1000
6.2.1     1000
6.2.2     1000
6.2.3     1000
6.3.1     1000
6.3.7     1000
7.1.1     1000
7.2.1     1000
7.3.1     1000
7.9.2     1000
8.1.4     1000
8.1.5     1000
8.2.1     1000
9.1.6     1000
9.3.1     1000
Name: count, dtype: int64


In [ ]:
# Create varied sets containing 3-5 different classes and each 20-200 reactions
import random

def create_varied_set()